# Profile Information
- Wildan Aziz Hidayat
- Undergraduate Students CompEng.
- 27 Mei 2005

# Check GPU is found or not

In [4]:
import torch

In [5]:
print("GPU found: ", torch.cuda.device_count())
print("GPU: ", torch.cuda.get_device_name())

GPU found:  1
GPU:  NVIDIA GeForce RTX 4050 Laptop GPU


# Import library

In [4]:
!git clone https://github.com/indobenchmark/indonlu.git

fatal: destination path 'indonlu' already exists and is not an empty directory.


In [5]:
%pip install transformers


     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ---------------------------------------- 43.7/43.7 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.4 MB 2.6 MB/s eta 0:00:04
   - -------------------------------------- 0.4/9.4 MB 4.5 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/9.4 MB 4.9 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/9.4 MB 6.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/9.4 MB 6.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.4/9.4 MB 5.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/9.4 MB 4.9 MB/s eta 0:00:02
   --------- ------------------------------ 2.3/9.4 MB 6.4 MB/s eta 0:00:02
   ----------- ---------------------------- 2.7/9.4 MB 6.6 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.4 MB 6.5 MB/s eta 0:00:01
   ------------- -

In [6]:
%pip install tqdm
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [17]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import defaultdict
import random

# read csv file

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from indonlu.utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader
from indonlu.utils.forward_fn import forward_sequence_classification
from indonlu.utils.metrics import document_sentiment_metrics_fn
from torch.utils.data import Dataset

In [4]:
df = pd.read_csv(r'/home/wildanaziz/TRAINING/bca-clean-sentiment.csv')
df

,merge_text,sentiment
0,pengguna baca mobile aktif aplikasi komplain d...,positive
1,coba update diperbaiki mudah langsung masuk ak...,positive
2,update lampu indikator merah keburu cek mutasi...,positive
3,membantu update ribet salah satunya dihilangka...,positive
4,ribet banget buka aplikasi izin lokasi nyalain...,negative
...,...,...
259306,super cepat,positive
259307,tap mantap,positive
259308,sip,neutral
259309,good application,positive


In [5]:
print(df['sentiment'].isnull().sum())

0


In [6]:
df

df.to_csv('dataset_clean.tsv', sep='\t', header=None, index=False)

# Data Splitting

In [13]:
train_df, valid_df = train_test_split(df, test_size=0.3, stratify=df.sentiment, random_state=42)
valid_df, test_df = train_test_split(valid_df, test_size=0.33, stratify=valid_df.sentiment, random_state=42)

In [14]:
print(f'Ukuran Train: {train_df.shape}')
print(f'Ukuran Validation: {valid_df.shape}')
print(f'Ukuran Test: {test_df.shape}')

Ukuran Train: (181517, 2)
Ukuran Validation: (52121, 2)
Ukuran Test: (25673, 2)


In [29]:
train_df.to_csv('train.tsv', sep='\t', header=None, index=False)
valid_df.to_csv('eval.tsv', sep='\t', header=None, index=False)
test_df.to_csv('test_df.tsv', sep='\t', header=None, index=False)

In [30]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

def count_param(module, trainable=False):
  if trainable:
    return sum(p.numel() for p in module.parameters() if p.requires_grad)
  else:
    return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']

def metrics_to_string(metric_dict):
  string_list = []
  for key, value in metric_dict.items():
    string_list.append('{}:{:.2f}'.format(key, value))
  return ', '.join(string_list)

set_seed(42)
print(set_seed)

<function set_seed at 0x0000023D0D2740E0>


In [31]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)
print(model)
print(count_param(model))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [32]:
merge_text = df.merge_text.values
tokenized_texts = tokenizer.tokenize(merge_text[3])

input_ids = []

for text in merge_text:
  encoded_text = tokenizer.encode(
      text,
      add_special_tokens=True,
  )
  input_ids.append(encoded_text)

print("Asli:", merge_text[3])
print("Tokenized:", tokenized_texts)
print("Input IDs:", input_ids[3])

Asli: membantu update ribet salah satunya dihilangkannya menu edit menghapus riwayat transaksi opsi menghapusnya mengganggu banyaknya inbox riwayat transaksi lihat mutasi tolong dibantu solusinya
Tokenized: ['membantu', 'update', 'ribet', 'salah', 'satunya', 'dihilangkan', '##nya', 'menu', 'edit', 'menghapus', 'riwayat', 'transaksi', 'opsi', 'menghapus', '##nya', 'mengganggu', 'banyaknya', 'inbox', 'riwayat', 'transaksi', 'lihat', 'mutasi', 'tolong', 'dibantu', 'solusinya']
Input IDs: [2, 1055, 4398, 11565, 427, 3040, 13325, 57, 1622, 5087, 6424, 6271, 3108, 7739, 6424, 57, 4826, 3642, 24935, 6271, 3108, 1173, 11949, 3854, 7517, 9149, 3]


In [33]:
optimizer = optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [22]:
import os
print(os.getcwd()) 

d:\TRAINING


In [35]:
train_dataset_path = 'D:/TRAINING/train.tsv'
valid_dataset_path = 'D:/TRAINING/eval.tsv'
test_dataset_path = 'D:/TRAINING/test_df.tsv'

In [36]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=2, shuffle=True)
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=2, shuffle=False)
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=32, num_workers=2, shuffle=False)

In [37]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'positive': 0, 'neutral': 1, 'negative': 2}
{0: 'positive', 1: 'neutral', 2: 'negative'}


# Training Validation and Test Deep Learning

In [38]:
n_epochs = 2
history = defaultdict(list)

for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))

    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

    # Evaluate Validation
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))

    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(EPOCH {}) VALID LOSS:{:4f} {}".format((epoch+1),
    total_loss/(i+1), metrics_to_string(metrics)))



  0%|          | 0/5673 [00:00<?, ?it/s]c:\Users\rifan\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
(Epoch 1) TRAIN LOSS:0.1918 LR:0.00000300: 100%|██████████| 5673/5673 [16:10<00:00,  5.84it/s]  


(Epoch 1) TRAIN LOSS:0.1918 ACC:0.93, F1:0.92, REC:0.92, PRE:0.93 LR:0.00000300


VALID LOSS:0.1088 ACC:0.96, F1:0.96, REC:0.95, PRE:0.96: 100%|██████████| 1629/1629 [09:36<00:00,  2.82it/s]


(Epoch 1) VALID LOSS:0.1088 ACC:0.96, F1:0.96, REC:0.95, PRE:0.96


VALID LOSS:0.1088 ACC:0.96, F1:0.96, REC:0.95, PRE:0.96: 100%|██████████| 1629/1629 [09:36<00:00,  2.82it/s]


...

(Epoch 2) TRAIN LOSS:0.0932 ACC:0.97, F1:0.96, REC:0.96, PRE:0.96 LR:0.00000300


VALID LOSS:0.0840 ACC:0.97, F1:0.97, REC:0.96, PRE:0.97: 100%|██████████| 1629/1629 [09:43<00:00,  2.79it/s]


(Epoch 2) VALID LOSS:0.0840 ACC:0.97, F1:0.97, REC:0.96, PRE:0.97


VALID LOSS:0.0840 ACC:0.97, F1:0.97, REC:0.96, PRE:0.97: 100%|██████████| 1629/1629 [10:12<00:00,  2.66it/s]


(EPOCH 2) VALID LOSS:0.083953 ACC:0.97, F1:0.97, REC:0.96, PRE:0.97


In [39]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df_pred = pd.DataFrame({'label':list_hyp}).reset_index()

100%|██████████| 803/803 [00:42<00:00, 19.11it/s]


# Cell inference

In [46]:
text = 'update aplikasinya aneh aktifin lokasi kadang pas login langkah memasukkan kartu kode otp beli pulsa kode otp masuk'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: update aplikasinya aneh aktifin lokasi kadang pas login langkah memasukkan kartu kode otp beli pulsa kode otp masuk | Label : negative (99.001%)


# Machine Learning

best accuracy train = 96%
best accuracy test = 92%

In [49]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Collecting tensorflow
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/385.2 MB 3.6 MB/s eta 0:01:46
   ---------------------------------------- 0.2/385.2 MB 2.0 MB/s eta 0:03:15
   ---------------------------------------- 0.4/385.2 MB 2.6 MB/s eta 0:02:31
   ---------------------------------------- 0.7/385.2 MB 3.4 MB/s eta 0:01:52
   ---------------------------------------- 0.8/385.2 MB 3.4 MB/s eta 0:01:55
   ---------------------------------------- 1.3/385.2 MB 4.6 MB/s eta 0:01:25
   ---------------------------------------- 1.4/385.2 MB 4.6 MB/s eta 0:01:24
   ---------------------------------------- 1.4/385.2 MB 4.6 MB/s eta 0:01:24
   ---------------------------------------- 2.0/385.2 MB 5.0 MB/s 

In [10]:
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = df['merge_text']
y = df['sentiment']

In [11]:
!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, make_scorer, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from sklearn.svm import SVC
import numpy as np

In [12]:
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

In [13]:
# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

,admin,akses,aktivasi,aman,aneh,apk,aplikasi,aplikasinya,app,atm,...,unable,update,upgrade,urusan,verifikasi,versi,via,video,wajah,wifi
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.164003,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.172538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.206837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.230219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.540307,0.0,0.0,0.0,0.0,0.184759,0.259006,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259306,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
259307,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
259308,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
259309,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((194483, 200), (64828, 200), (194483,), (64828,))

In [15]:
# Memeriksa ukuran setelah split
print("Ukuran X_train_tfidf:", X_train.shape)
print("Ukuran X_test_tfidf:", X_test.shape)

Ukuran X_train_tfidf: (194483, 200)
Ukuran X_test_tfidf: (64828, 200)


In [16]:
# Mengatasi imbalance data dengan SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
X_train_res.shape, y_train_res.shape

((295053, 200), (295053,))

In [59]:
# Membuat model Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Menggunakan StratifiedKFold untuk memastikan distribusi kelas seimbang pada setiap fold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Melatih model pada seluruh data pelatihan
rf_classifier.fit(X_train_res, y_train_res)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train = rf_classifier.predict(X_train_res)
y_pred_test = rf_classifier.predict(X_test)

# Evaluasi akurasi model
accuracy_train = accuracy_score(y_train_res, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

# Menampilkan akurasi dan laporan klasifikasi
print('Random Forest - accuracy_train:', accuracy_train)
print('Random Forest - accuracy_test:', accuracy_test)
print(classification_report(y_test, y_pred_test))

Random Forest - accuracy_train: 0.9657858079734827
Random Forest - accuracy_test: 0.904917628185352
              precision    recall  f1-score   support

    negative       0.87      0.89      0.88     17521
     neutral       0.84      0.92      0.88     14205
    positive       0.96      0.91      0.93     33102

    accuracy                           0.90     64828
   macro avg       0.89      0.91      0.90     64828
weighted avg       0.91      0.90      0.91     64828



In [27]:
# Inisialisasi model Logistic Regression
logistic_regression = LogisticRegression(random_state=42, max_iter=1000)

# Membuat objek KFold
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# Melakukan cross-validation
scores = cross_val_score(logistic_regression, X_train_res.toarray(), y_train_res, cv=kfold, scoring='accuracy')

# Mencetak hasil cross-validation
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {scores.mean()}')

# Melatih model pada seluruh data training
logistic_regression.fit(X_train_res.toarray(), y_train_res)

# Prediksi dengan model
y_pred_train_lr = logistic_regression.predict(X_train_res.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi model
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train_res)
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)
report_test_lr = classification_report(y_test, y_pred_test_lr)

print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)
print('Classification Report:')
print(report_test_lr)

Cross-validation scores: [0.91149048 0.9089689  0.91002633]
Mean cross-validation score: 0.9101619031157114
Logistic Regression - accuracy_train: 0.9112532324700986
Logistic Regression - accuracy_test: 0.9102085518603072
Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.90      0.89     17521
     neutral       0.83      0.91      0.87     14205
    positive       0.96      0.92      0.94     33102

    accuracy                           0.91     64828
   macro avg       0.89      0.91      0.90     64828
weighted avg       0.91      0.91      0.91     64828



In [26]:
# Membuat model SVM dengan parameter default
svm = SVC(kernel='linear', random_state=42)

# Melatih model pada seluruh data training
svm.fit(X_train_res, y_train_res)

# Prediksi dengan model
y_pred_train = svm.predict(X_train_res)
y_pred = svm.predict(X_test)

# Evaluasi model
accuracy_train = accuracy_score(y_train_res, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Akurasi train: {accuracy_train}')
print(f'Akurasi test: {accuracy_test}')
print('Classification Report:')
print(report)

Cross-validation scores: [0.91397139 0.91143964 0.91290378]
Mean cross-validation score: 0.9127716037457677
Akurasi train: 0.913700250463476
Akurasi test: 0.9128308755476029
Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.90      0.89     17521
     neutral       0.84      0.92      0.88     14205
    positive       0.97      0.92      0.94     33102

    accuracy                           0.91     64828
   macro avg       0.90      0.91      0.90     64828
weighted avg       0.92      0.91      0.91     64828



In [30]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
  Using cached xgboost-2.1.0-py3-none-manylinux_2_28_x86_64.whl (153.9 MB)


In [34]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Mengonversi label menjadi angka
label_encoder = LabelEncoder()
y_train_res_encoded = label_encoder.fit_transform(y_train_res)
y_test_encoded = label_encoder.transform(y_test)

# Membuat model XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)

# Melatih model pada seluruh data training
xgb_model.fit(X_train_res, y_train_res_encoded)

# Prediksi dengan model
y_pred_train = xgb_model.predict(X_train_res)
y_pred = xgb_model.predict(X_test)

# Mengonversi kembali prediksi menjadi label asli
y_pred_train_decoded = label_encoder.inverse_transform(y_pred_train)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Evaluasi model
accuracy_train = accuracy_score(y_train_res, y_pred_train_decoded)
accuracy_test = accuracy_score(y_test, y_pred_decoded)
report = classification_report(y_test, y_pred_decoded)

print(f'Akurasi train: {accuracy_train}')
print(f'Akurasi test: {accuracy_test}')
print('Classification Report:')
print(report)


Akurasi train: 0.9195195439463418
Akurasi test: 0.9064447460973654
Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.89      0.89     17521
     neutral       0.81      0.93      0.87     14205
    positive       0.96      0.91      0.93     33102

    accuracy                           0.91     64828
   macro avg       0.89      0.91      0.90     64828
weighted avg       0.91      0.91      0.91     64828



In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Membuat model individual
logreg = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)
svm = SVC(kernel='linear', random_state=42, probability=True)

# Membuat Voting Classifier
voting_clf = VotingClassifier(
    estimators=[
        ('logreg', logreg),
        ('rf', rf),
        ('svm', svm)
    ],
    voting='soft'  # Menggunakan probabilitas rata-rata untuk voting
)

# Melatih Voting Classifier pada seluruh data training
voting_clf.fit(X_train_res, y_train_res)

# Prediksi dengan model
y_pred_train = voting_clf.predict(X_train_res)
y_pred = voting_clf.predict(X_test)

# Evaluasi model
accuracy_train_voting = accuracy_score(y_train_res, y_pred_train)
accuracy_test_voting = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Akurasi train: {accuracy_train_voting}')
print(f'Akurasi test: {accuracy_test_voting}')
print('Classification Report:')
print(report)


Akurasi train: 0.933947460286796
Akurasi test: 0.9151138396988956
Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.90      0.89     17521
     neutral       0.84      0.93      0.88     14205
    positive       0.97      0.92      0.94     33102

    accuracy                           0.92     64828
   macro avg       0.90      0.91      0.91     64828
weighted avg       0.92      0.92      0.92     64828



In [22]:
accuracy_train_percent = round(accuracy_train_voting * 100)
accuracy_test_percent = round(accuracy_test_voting * 100)

print(f'Akurasi train: {accuracy_train_percent}%')
print(f'Akurasi test: {accuracy_test_percent}%')

Akurasi train: 93%
Akurasi test: 92%
